<h3>FP2 Dataset</h3>

#### Two sources of data for each company:
<ol>
    <li>Text fom their financial reports</li>
    <li>Time series of their stock price, volume, etc</li>
</ol>

### Tickers

In [293]:
import re
import requests as req

CIK_TICKER = {}
# Creates a dictionary mapping cik to tickers
# using scraped sec data
def createTickerDict():
    r = req.get('https://www.sec.gov/include/ticker.txt',stream=True)
    for line in r.iter_lines():
        file_line = line.decode("utf-8").split("\t")
        CIK_TICKER[file_line[1]] = file_line[0]
createTickerDict()

### Prices

Full URL: https://api.koyfin.com/api/v2/commands/g/g.eq/AMZN?dateFrom=1900-10-15&dateTo=2020-10-30&period=daily

Base URL: https://api.koyfin.com/api/v2/commands/g/g.eq/

ticker + "?dateFrom=" + Year-month-day + "&dateTo=" + Year-month-day + "&period=daily"

In [294]:
# Return json dictionary of pricing information given valid ticker
def request_json(ticker: str) -> dict:
    url_base = "https://api.koyfin.com/api/v2/commands/g/g.eq/"
    
    # for some reason ticker such as BRK-A, appears as BRK.A in url
    ticker = ticker.replace("-", ".")
    
    # start and end date can be changed to get less data
    # period = yearly, monthly, weekly, or daily also can be changed
    #default_date = "?dateFrom=1900-10-15&dateTo=2020-10-30&period=monthly"
    default_date = "?dateFrom=2015-10-15&dateTo=2020-10-30&period=monthly"
    
    r = req.get(url_base + ticker + default_date)
    #print("\nRequest Response {}\n".format(r.status_code)) debugging 404
    return r.json()

In [295]:
import pandas as pd

# given ticker returns dataframe of available pricing info, monthly starting 2015
def json_df(ticker: str):
    json_data = request_json(ticker)
    json_prices = json_data["graph"]["data"]
    truncated_prices = [date[0:6] for date in json_prices]

    column_label = ["Date", "Open", "High", "Low", "Close/Last", "Volume"]

    df = pd.DataFrame(truncated_prices, columns=column_label) 
    df.insert(0,"Ticker","")
    df["Ticker"] = json_data["ticker"]
    return df

In [298]:
json_df("AAPL").head()

,Ticker,Date,Open,High,Low,Close/Last,Volume
0,AAPL,2015-10-30,27.2675,30.3050,26.8275,29.8750,4454815240
1,AAPL,2015-11-30,29.9675,30.9550,27.7500,29.5750,3002562360
2,AAPL,2015-12-31,29.6875,29.9650,26.2050,26.3150,3690505120
3,AAPL,2016-01-29,25.6525,26.4625,23.0975,24.3350,5087392560
4,AAPL,2016-02-29,24.1175,24.7225,23.1475,24.1725,3243450200


### Text of Reports

In [299]:
import unicodedata
import re
from bs4 import BeautifulSoup as BS
from xml.dom import minidom  

# parsing some reports here takes 20 and more seconds
def getStatement(endOfURL: str):
    """
    gets the text in a 10k/10q statement
    :param endOfURL: the end of the url to the statement file
    : endofURL is found from masterfile
    :return: list of text of the statement
    """
    # making the link
    base = 'https://www.sec.gov/Archives/'
    fullUrl = base + endOfURL
    
    # souping the site
    r = req.get(fullUrl).content.decode("latin-1")
    
    # uncomment for 20x faster but no parsing and taking up too much RAM
    #return r #
    
    soup = BS(r, 'html.parser')

    # filtering the contents of the site
    text = []
    for x in soup.find_all('p')[:-1]:
        elm = BS(str(x), 'html.parser').get_text()

        # removing links to references
        # still the disclosures at the bottom
        if re.search(r'Reference [1-9]: http://', elm):
            continue

        elm = unicodedata.normalize("NFKD", elm)

        # replacing all white space characters for some reason I could not get \s to work
        elm = re.sub(r'[ \t\n\r\f\v]', ' ', elm)

        # 2 or more spaces get replaced to one space
        elm = re.sub(r' {2,}', '', elm)

        # if its one string then we dont care
        if len(elm) > 1:
            text.append(elm)

    return text # return as single string, remove join to return as list

In [302]:
# example statement call (truncated)
getStatement("/edgar/data/1002047/0001564590-15-000944.txt")[:20]

['UNITED STATES ',
 'SECURITIES AND EXCHANGE COMMISSION ',
 'Washington, D.C. 20549 ',
 'Form 10-Q ',
 '(Mark One) ',
 'QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 ',
 'For the quarterly period ended January 23, 2015 ',
 'or ',
 ' ̈',
 'TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 ',
 'For the transition period fromto',
 'Commission File Number 000-27130 ',
 'NetApp, Inc. ',
 '(Exact name of registrant as specified in its charter) ',
 'Delaware',
 '77-0307520',
 '(State or other jurisdiction of',
 '(I.R.S. Employer',
 'incorporation or organization)',
 'Identification No.)']

In [303]:
# scrapes the master file as a list of lines from the SEC
# containing links to all available financial reports
# for a given year and quarter
def read_master(year:str, qtr:str) -> list:
    lines = []
    base_url = 'https://www.sec.gov/Archives/edgar/full-index/'
    r = req.get(base_url + year + "/" + qtr + "/master.idx", stream=True)
    
    reading = False
    for line in r.iter_lines():
        if line:
            if reading:
                line = line.decode("latin-1").split("|")
                if line[2] == "10-Q" or line[2] == "10-K":
                    lines.append(line)
            # ignore the heading and only read lines after --------... seperator
            if str(line).count("-") > 10:
                reading = True
    return lines

In [305]:
# example scraping master file (truncated)
read_master("2020", "QTR1")[:10]

[['1000045',
  'NICHOLAS FINANCIAL INC',
  '10-Q',
  '2020-02-14',
  'edgar/data/1000045/0001564590-20-004703.txt'],
 ['1000209',
  'MEDALLION FINANCIAL CORP',
  '10-K',
  '2020-03-30',
  'edgar/data/1000209/0001564590-20-014310.txt'],
 ['1000228',
  'HENRY SCHEIN INC',
  '10-K',
  '2020-02-20',
  'edgar/data/1000228/0001000228-20-000018.txt'],
 ['1000229',
  'CORE LABORATORIES N V',
  '10-K',
  '2020-02-10',
  'edgar/data/1000229/0001564590-20-004075.txt'],
 ['1000230',
  'OPTICAL CABLE CORP',
  '10-K',
  '2020-01-27',
  'edgar/data/1000230/0001437749-20-001224.txt'],
 ['1000230',
  'OPTICAL CABLE CORP',
  '10-Q',
  '2020-03-16',
  'edgar/data/1000230/0001437749-20-005327.txt'],
 ['1000232',
  'KENTUCKY BANCSHARES INC /KY/',
  '10-K',
  '2020-03-10',
  'edgar/data/1000232/0001558370-20-002327.txt'],
 ['1000298',
  'IMPAC MORTGAGE HOLDINGS INC',
  '10-K',
  '2020-03-13',
  'edgar/data/1000298/0001558370-20-002683.txt'],
 ['1000623',
  'SCHWEITZER MAUDUIT INTERNATIONAL INC',
  '10-K',
 

### Text of All Reports to DataFrame

In [ ]:
import time

# take all available reports from 2015-2020 from all 4q
YEARS = ['2015', '2016', '2017', '2018', '2019', '2020']
QTRS = ['QTR1', 'QTR2', 'QTR3', 'QTR4']

df = pd.DataFrame(columns = ["Ticker", "ReportDate", "ReportType", "Text"])

for yr in YEARS:
    for q in QTRS:
        print("Begin {}, {}".format(yr, q))
        reports = read_master(yr, q)
        #ex. ["1192448", "GLAUKOS Corp", "10-K", "2019-04-03", "edgar/data/1192448/0001104659-19-019728.txt"]
        for report in reports:
            report_type = report[2]
            # filter the reports we want (there are ALOT)
            if (report_type == "10-Q" or report_type == "10-K"):
                report_url = report[4]
                report_date = report[3]
                company_name = report[1]
                cik = report[0]
                
                # arbitrary cik limit to limit how many companies are chosen
                if CIK_TICKER.get(cik) != None and int(cik) < 1100682:
                    print("\tGetting report for {}".format(company_name))
                    ticker = CIK_TICKER[cik]
                    
                    time.sleep(.2) # avoid rate limit?
                    print("\t\tGetting {} at {}".format(report_type, report_url))
                    text = getStatement(report_url)
                    
                    df = df.append({"Ticker":ticker.upper(), "ReportDate":report_date,
                               "ReportType": report_type, "Text": text}, ignore_index=True)
                    print("\t\tReport Added")
df

Begin 2015, QTR1
	Getting report for NICHOLAS FINANCIAL INC
		Getting 10-Q at edgar/data/1000045/0001193125-15-038970.txt
		Report Added
	Getting report for MEDALLION FINANCIAL CORP
		Getting 10-K at edgar/data/1000209/0001193125-15-087622.txt


In [ ]:
df

In [ ]:
df.to_csv("reports.csv")